<a href="https://colab.research.google.com/github/farrelrassya/FeatureSelectionforMachineLearning/blob/main/01.Constant-Quasi-Constant-Duplicates/03.Duplicated_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Duplicated features
Often datasets contain duplicated features, that is, features that despite having different names, are identical.

In addition, we may often introduce duplicated features when performing one hot encoding of categorical variables, particularly if our datasets have many and /or highly cardinal categorical variables.

Identifying and removing duplicated, and therefore redundant features, is an easy first step towards feature selection and more interpretable machine learning models.

Here I will demonstrate how to identify duplicated features using a dataset that I created for this course.

There is no function in Pandas to find duplicated columns. So we need to write a bit code to do so.

Note Finding duplicated features can be a computationally costly operation in Python, therefore depending on the size of your dataset, you might not always be able to do it.

This method that I describe here to find duplicated features works for both numerical and categorical variables.

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('dataset_1.csv')
data.shape

(50000, 301)

In [ ]:
[col for col in data.columns if data[col].isnull().sum() > 0]

[]

**Important**

In all feature selection procedures, it is good practice to select the features by examining only the training set. And this is to avoid overfit.

In [ ]:
# separate dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1), # drop the target
    data['target'], # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 300), (15000, 300))

**Remove constant and quasi-constant**

In [ ]:
# remove constant and quasi-constant features first:
# we can remove the 2 types of features together with this code
# (we used it in our previous notebook)

# create an empty list
quasi_constant_feat = []

# iterate over every feature
for feature in X_train.columns:

    # find the predominant value, that is the value that is shared
    # by most observations
    predominant = (X_train[feature].value_counts() / float(
        len(X_train))).sort_values(ascending=False).values[0]

    # evaluate predominant feature: do more than 99% of the observations
    # show 1 value?
    if predominant > 0.998:
        quasi_constant_feat.append(feature)

len(quasi_constant_feat)

142

In [ ]:
# we can then drop these columns from the train and test sets:

X_train.drop(labels=quasi_constant_feat, axis=1, inplace=True)
X_test.drop(labels=quasi_constant_feat, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 158), (15000, 158))

**Remove duplicated features**
To identify duplicated variables we need to iterate through all features of our dataset, and for each and every feature, try and find others that are identical, or duplicates.

We will create a dictionary of {variable: duplicated variables} pairs to identify them more easily throughout the demo. Keep in mind that in a dataset, there could be 2 or more features that are identical to each other.

In [ ]:
# check for duplicated features in the training set:

# create an empty dictionary, where we will store
# the groups of duplicates
duplicated_feat_pairs = {}

# create an empty list to collect features
# that were found to be duplicated
_duplicated_feat = []


# iterate over every feature in our dataset:
for i in range(0, len(X_train.columns)):

    # this bit helps me understand where the loop is at:
    if i % 10 == 0:
        print(i)

    # choose 1 feature:
    feat_1 = X_train.columns[i]

    # check if this feature has already been identified
    # as a duplicate of another one. If it was, it should be stored in
    # our _duplicated_feat list.

    # If this feature was already identified as a duplicate, we skip it, if
    # it has not yet been identified as a duplicate, then we proceed:
    if feat_1 not in _duplicated_feat:

        # create an empty list as an entry for this feature in the dictionary:
        duplicated_feat_pairs[feat_1] = []

        # now, iterate over the remaining features of the dataset:
        for feat_2 in X_train.columns[i + 1:]:

            # check if this second feature is identical to the first one
            if X_train[feat_1].equals(X_train[feat_2]):

                # if it is identical, append it to the list in the dictionary
                duplicated_feat_pairs[feat_1].append(feat_2)

                # and append it to our monitor list for duplicated variables
                _duplicated_feat.append(feat_2)

                # done!

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


In [ ]:
# let's explore our list of duplicated features
len(_duplicated_feat)

6

In [ ]:
# these are the ones:

_duplicated_feat

['var_148', 'var_199', 'var_296', 'var_250', 'var_232', 'var_269']

In [ ]:
# let's explore the dictionary we created:

duplicated_feat_pairs

{'var_4': [],
 'var_5': [],
 'var_8': [],
 'var_13': [],
 'var_15': [],
 'var_17': [],
 'var_18': [],
 'var_19': [],
 'var_21': [],
 'var_22': [],
 'var_25': [],
 'var_26': [],
 'var_27': [],
 'var_29': [],
 'var_30': [],
 'var_31': [],
 'var_35': [],
 'var_37': ['var_148'],
 'var_38': [],
 'var_41': [],
 'var_46': [],
 'var_47': [],
 'var_49': [],
 'var_50': [],
 'var_51': [],
 'var_52': [],
 'var_54': [],
 'var_55': [],
 'var_57': [],
 'var_58': [],
 'var_62': [],
 'var_63': [],
 'var_64': [],
 'var_68': [],
 'var_70': [],
 'var_74': [],
 'var_75': [],
 'var_76': [],
 'var_79': [],
 'var_82': [],
 'var_83': [],
 'var_84': ['var_199'],
 'var_85': [],
 'var_86': [],
 'var_88': [],
 'var_91': [],
 'var_93': [],
 'var_94': [],
 'var_96': [],
 'var_100': [],
 'var_101': [],
 'var_103': [],
 'var_105': [],
 'var_107': [],
 'var_108': [],
 'var_109': [],
 'var_110': [],
 'var_114': [],
 'var_117': [],
 'var_118': [],
 'var_119': [],
 'var_121': [],
 'var_123': [],
 'var_128': [],
 'var_131'

We see that for every feature, if it had duplicates, we have entries in the list, otherwise, we have empty lists. Let's explore those features with duplicates now:

In [ ]:
# let's explore the number of keys in our dictionary

# we see it is 152, because 6 of the 158 were duplicates,
# so they were not included as keys

print(len(duplicated_feat_pairs.keys()))

152


In [ ]:
# print the features with its duplicates

# iterate over every feature in our dict:
for feat in duplicated_feat_pairs.keys():

    # if it has duplicates, the list should not be empty:
    if len(duplicated_feat_pairs[feat]) > 0:

        # print the feature and its duplicates:
        print(feat, duplicated_feat_pairs[feat])
        print()

var_37 ['var_148']

var_84 ['var_199']

var_143 ['var_296']

var_177 ['var_250']

var_226 ['var_232']

var_229 ['var_269']



In [ ]:
# let's check that indeed those features are duplicated
# I select a pair from above

X_train[['var_37', 'var_148']].head(10)

,var_37,var_148
17967,0,0
32391,0,0
9341,0,0
7929,0,0
46544,0,0
4149,0,0
33426,0,0
3002,0,0
6974,0,0
16864,0,0


In [ ]:
X_train['var_37'].unique()

array([ 0,  3,  6,  9, 12, 21, 33, 15])

In [ ]:
X_train['var_148'].unique()

array([ 0,  3,  6,  9, 12, 21, 33, 15])

In [ ]:
# let's explore parts of the dataframe where the values in
# these features are different from 0:

X_train[X_train['var_37'] != 0][['var_37', 'var_148']].head(10)

,var_37,var_148
37493,3,3
20251,6,6
4264,6,6
48480,3,3
31607,3,3
41172,3,3
13502,3,3
7759,3,3
46118,3,3
2638,3,3


As we see, these features are indeed identical :)

There are more duplicated features in this dataset, which we removed because they are constant or quasi-constant. So as an exercise, why don't you try and find which additional features in this dataset are duplicates?

In [ ]:
# finally, to remove the duplicates, what we are going to do is to retain
# the keys of the dictionary

# do you understand why? if not, go back to our loop in cell 7 and try to
# determine the reason

X_train = X_train[duplicated_feat_pairs.keys()]
X_test = X_test[duplicated_feat_pairs.keys()]

X_train.shape, X_test.shape

((35000, 152), (15000, 152))

We can see how we further reduced our dataset by 6 additional features.

In summary, by removing constant, quasi-constant and duplicated features, we reduced our original 300 feature dataset to a 152 feature dataset. Well done us!

That is all for this lecture, I hope you enjoyed it and see you in the next one!